In [3]:
import numpy as np
import scanpy as sc
import anndata as ad
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import plotnine as pln

# Import Data

In [30]:
# load data if you don't want to run the whole thing
adata = ad.read_h5ad(filename="/data/rudensky/EYW/SIG04_240816/python_outs/SIG04_merge_singlets_ZscoreNorm_scanpy.h5ad")

In [34]:
adata = adata[adata.obs.gem_group == "lane2"]

In [7]:
adata.obs["p139_expression"] = sc.get.obs_df(adata,"p139-T7oBC5p-MS2")

In [37]:
temp = sc.get.aggregate(adata,"oBC_feature_call","mean")

In [8]:
#sc.pp.regress_out(adata,keys="p139_expression")

In [9]:
# import functions from perturbseq codebase
import sys
sys.path.insert(0, '/data/rudensky/EYW/git_projects/SIG04_240816/')
from perturbseq import *

In [35]:
# only include genes expressed in > 5% of cells
testGenes = adata.var[adata.var['n_cells'] > 2722].index.tolist()

KSS, pval, pval_adj = ks_de(adata, key='oBC_feature_call',
      genes=testGenes,
      control_cells='oBC_feature_call == "p129"',
      normalized=True,
      multi_method='fdr_bh',
      n_jobs=24)

3642 control cells


/data/rudensky/EYW/git_projects/SIG04_240816/perturbseq/differential_expression.py:141: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:   46.3s
[Parallel(n_jobs=24)]: Done  13 tasks      | elapsed:   47.9s
[Parallel(n_jobs=24)]: Done  24 tasks      | elapsed:   50.0s
[Parallel(n_jobs=24)]: Done  45 out of  83 | elapsed:  1.3min remaining:  1.1min
[Parallel(n_jobs=24)]: Done  54 out of  83 | elapsed:  1.4min remaining:   46.3s
[Parallel(n_jobs=24)]: Done  63 out of  83 | elapsed:  1.5min remaining:   28.7s
[Parallel(n_jobs=24)]: Done  72 out of  83 | elapsed:  1.7min remaining:   15.2s
[Parallel(n_jobs=24)]: Done  81 out of  83 | elapsed:  1.8min remaining:    2.7s
[Paralle

In [27]:
KSS.index = testGenes
pval.index = testGenes
pval_adj.index = testGenes

In [28]:
# Assuming pval_adj is a DataFrame
below_threshold = (pval_adj < 0.05).sum(axis=0)

# Filter column names where the count is greater than 0
columns_below_threshold = below_threshold[below_threshold > 0].index

# Print the column names and their associated counts
for col in columns_below_threshold:
    print(f"Column: {col}, Count below threshold: {below_threshold[col]}")

Column: ADIPOQ, Count below threshold: 21
Column: BMP4, Count below threshold: 33
Column: BMP7, Count below threshold: 6
Column: BMP10, Count below threshold: 7
Column: CCL2, Count below threshold: 5289
Column: CCL4, Count below threshold: 2
Column: CCL5, Count below threshold: 1195
Column: CCL7, Count below threshold: 27
Column: CCL8, Count below threshold: 17
Column: CCL12, Count below threshold: 13
Column: CCL19, Count below threshold: 1980
Column: CCL21A, Count below threshold: 3055
Column: CCL25, Count below threshold: 626
Column: CXCL9, Count below threshold: 6
Column: CXCL10, Count below threshold: 10
Column: CXCL11, Count below threshold: 121
Column: CXCL12, Count below threshold: 2
Column: CXCL13, Count below threshold: 1
Column: CXCL16, Count below threshold: 18
Column: GDF2, Count below threshold: 160
Column: GDF7, Count below threshold: 2
Column: GDF11, Count below threshold: 3911
Column: GDF15, Count below threshold: 1
Column: IFNA, Count below threshold: 1233
Column: IFNA

In [77]:
# Specify the column you're interested in
column_name = 'IL4'

# Check if the column exists in the DataFrame
if column_name in pval_adj.columns:
    # Get the boolean Series where p-values are below 0.05
    below_threshold = pval_adj[column_name] < 0.05
    
    # Get the row names (indices) where the condition is True
    rows_below_threshold = pval_adj.index[below_threshold]
    
    # Print the row names
    print(f"Row names for column '{column_name}' with p-values below 0.05:")
    for row in rows_below_threshold:
        print(row)
else:
    print(f"Column '{column_name}' does not exist in the DataFrame.")

Row names for column 'IL4' with p-values below 0.05:
Stat4
Hspd1
Ctla4
Rassf5
Niban1
Gata3
Rxra
Cytip
Nat10
Nop56
Prnp
Id1
Pdrg1
Dynlrb1
Ncoa3
Il2
1110032F04Rik
Crabp2
Txnip
Fam241a
Lef1
B4galt1
Tmem245
Eif2b3
Plk3
Ak2
Wasf2
Capzb
Sdf4
Tnfrsf18
Tes
Fam3c
Gprin3
Tmsb10
Lrig1
Ccnd2
Irag2
Etfb
Abhd17c
Rhog
Lsp1
Atp11a
Dusp4
Jund
Rab8b
Plscr1
Cish
Glb1
Ccr4
Marcks
Ddt
Sptbn1
Lcp2
Psme2b
Shmt1
Grap
Trp53
Eno3
Ccr7
Stat5a
Pitpnc1
Prkca
Actn1
Klf6
Aopep
Psme2
Rgcc
Mtdh
Pdgfb
Tfap4
B630019A10Rik
Btla
Cd200
Cd96
Ifngr2
Tagap
Capn15
Stk19
Clic1
Zfp318
Egr1
Malt1
Ms4a4b
Ms4a6b
Cd274
Ints6l
Arhgef6
Itm2a
Prps1
Tmsb4x
p139-T7oBC5p-MS2
